### 함수 저장소

1. 학습 데이터셋 비교 가시화

visualize_multiple_dataloaders([test_loader], test_loader, images_per_loader=5)

2. 실험 진행용 학습모델 저장

### Neural Network Master Switch Board

In [ ]:
# Neural Network Retrain Option
#모델/데이터 전체 리셋
Data_Reset = False
#학습된 모델 없등면 자동으로 학습
Auto_Init = True
defualt_epoch = 5
#Model Save Path
default_path = "default_path_less_corrupt_Maintanence"
#로그 기록 여부
log_save = True
## Switch Baord based on stage
poision_sys_reset = False
restore_sys_reset = False
reclass_sys_reset = True
multiplier_sys_reset = False
##Muliplier_sys_reset by Type
M_reset_auto_10 =False
M_reset_unet_10 =False
M_reset_auto_attacked =False
M_reset_unet_attacked =False
M_reset_auto_100 =False
M_reset_unet_100 = False
# CUDA 이용 GPU 번호 입력 (Default : 0)
GPU_NUM = 1

### 1. 라이브러리 불러오기

In [ ]:
import os
import shutil
import random
from torch.utils.data import DataLoader
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import gc


import torchvision
from torchvision.datasets import CIFAR10
from torchvision.datasets import CIFAR100
from torchvision.transforms import ToTensor
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torchvision.transforms as transforms
from PIL import Image
from torchvision import transforms


# Visualization
import cv2
import matplotlib.pyplot as plt
from torchsummary import summary

#t-SNE
from sklearn.manifold import TSNE

#train function
from torch.utils.data import DataLoader

In [ ]:
if torch.cuda.is_available():
    if GPU_NUM > (torch.cuda.device_count() - 1):
        GPU_NUM = torch.cuda.device_count() - 1
    device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
    torch.cuda.set_device(device) # change allocation of current GPU
    print('Current cuda device:', torch.cuda.current_device())
    print('Count of using GPUs:', torch.cuda.device_count())
else:
    device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

##### Automated Git Ignore 

In [ ]:
def gitignore(directory = default_path, gitignore_path=".gitignore"):

    # .gitignore 파일이 존재하는지 확인
    if not os.path.exists(gitignore_path):
        print(f"{gitignore_path} 파일이 존재하지 않습니다. 새로 생성합니다.")
        with open(gitignore_path, 'w') as f:
            f.write("")  # 빈 .gitignore 파일 생성
    
    # .gitignore 파일을 읽기
    with open(gitignore_path, 'r') as f:
        lines = f.readlines()

    # 이미 디렉토리가 .gitignore에 있는지 확인
    ignore_line = f"{directory}/\n"
    if ignore_line not in lines:
        # 디렉토리가 없으면 추가
        with open(gitignore_path, 'a') as f:
            f.write(ignore_line)
        print(f"{directory} 디렉토리를 .gitignore에 추가했습니다.")
    else:
        print(f"{directory} 디렉토리가 이미 .gitignore에 존재합니다.")


### 2. 데이터셋 정의 - 비오염

In [ ]:
# Data loading and transformations
transform = transforms.Compose([
  transforms.RandomHorizontalFlip(),
  transforms.RandomCrop(32, padding=4),
  transforms.ToTensor(),
  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
# Use PyTorch's torchvision.transforms.ToTenesor() to convert the dataset to tensor format
train_set = CIFAR10(root='./data', train=True, download=True, transform=transform)
test_set = CIFAR10(root='./data', train=False, download=True, transform=transform)
add_set = CIFAR100(root='./data', train=True, download=True, transform=transform)

In [ ]:
# Use PyTorch's DataLoader to divide the dataset into mini-batches and load the data
train_loader = DataLoader(dataset=train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=64, shuffle=False)
add_loader = DataLoader(dataset=add_set, batch_size=64, shuffle=False)

In [ ]:
def visualize_multiple_dataloaders(dataloaders, trainloader, images_per_loader=5):
    num_loaders = len(dataloaders)
    classes = trainloader.dataset.classes

    # 플롯 크기 설정
    plt.figure(figsize=(8,8))

    for i, loader in enumerate(dataloaders):
        # 각 DataLoader에서 배치 하나 가져오기
        batch = next(iter(loader))
        if len(batch) == 2:
            images, labels = batch
        elif len(batch) == 4:\
            _ , _, images, labels = batch
        images = torch.clamp(images, 0, 1)  # 이미지를 0과 1 사이로 클리핑하여 표시 문제 방지

        # 각 DataLoader에서 선택한 이미지 수만큼 시각화
        for j in range(images_per_loader):
            idx = i * images_per_loader + j
            plt.subplot(num_loaders, images_per_loader, idx + 1)
            plt.imshow(images[j].permute(1, 2, 0))  # 이미지 차원 변경: (C, H, W) -> (H, W, C)
            plt.title(classes[labels[j].item()])
            plt.axis('off')

    plt.tight_layout()
    plt.show()

### 3. 신경망 코딩

#### 3.1. ResNet50

In [ ]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        layers = []
        layers.append(block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels
        for _ in range(1, num_blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out

#### 3.2. Autoencoder 모델 정의

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        
        # Encoder 부분
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),   # 32x32 -> 16x16
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),  # 16x16 -> 8x8
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1), # 8x8 -> 4x4
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(256 * 4 * 4, 1024),
            nn.ReLU(),
            nn.Linear(1024, 128)  # latent space (압축된 표현)
        )
        
        # Decoder 부분
        self.decoder = nn.Sequential(
            nn.Linear(128, 1024),
            nn.ReLU(),
            nn.Linear(1024, 256 * 4 * 4),
            nn.ReLU(),
            nn.Unflatten(1, (256, 4, 4)),
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1), # 4x4 -> 8x8
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),   # 8x8 -> 16x16
            nn.ReLU(),
            nn.ConvTranspose2d(64, 3, kernel_size=3, stride=2, padding=1, output_padding=1),     # 16x16 -> 32x32
            nn.Tanh()  # 0~1 범위로 출력
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

#### 3.3. U-Net 모델 정의

In [ ]:
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        
        # 인코더 (Contracting Path)
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)  # 32x32 -> 16x16
        )
        
        self.encoder_2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)  # 16x16 -> 8x8
        )
        
        self.encoder_3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)  # 8x8 -> 4x4
        )

        # 병목층 (Bottleneck)
        self.bottleneck = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU()
        )

        # 디코더 (Expanding Path)
        self.decoder_1 = nn.Sequential(
            nn.Conv2d(512, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='nearest')  # 4x4 -> 8x8
        )
        
        self.decoder_2 = nn.Sequential(
            nn.Conv2d(256, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='nearest')  # 8x8 -> 16x16
        )
        
        self.decoder_3 = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='nearest')  # 16x16 -> 32x32
        )
        
        # 최종 출력 레이어
        self.final_conv = nn.Conv2d(64, 3, kernel_size=3, padding=1)  # 출력 채널 3 (RGB 이미지)

    def forward(self, x):
        # 인코더
        enc1 = self.encoder(x)
        enc2 = self.encoder_2(enc1)
        enc3 = self.encoder_3(enc2)
        
        # 병목층
        bottleneck = self.bottleneck(enc3)
        
        # 디코더
        dec1 = self.decoder_1(bottleneck)
        dec2 = self.decoder_2(dec1)
        dec3 = self.decoder_3(dec2)
        
        # 최종 출력
        out = self.final_conv(dec3)
        return out


#### 3.4 T-SNE

In [ ]:
class TSNE_Model(torch.nn.Module):
    def __init__(self, n_components=2, perplexity=30.0, n_iter=1000):
        super(TSNE_Model, self).__init__()
        self.n_components = n_components
        self.perplexity = perplexity
        self.n_iter = n_iter

    def forward(self, X):
        """
        X: (N, D) tensor, where N is the number of data points and D is the dimension of each data point
        """
        X = X.detach().cpu().numpy()  # PyTorch tensor -> NumPy array
        tsne = TSNE(n_components=self.n_components, perplexity=self.perplexity, n_iter=self.n_iter)
        tsne_results = tsne.fit_transform(X)
        return torch.tensor(tsne_results, dtype=torch.float32).to(X.device)


#### 3.5 NLM

In [ ]:
class NLM_Model(torch.nn.Module):
    def __init__(self, patch_size=3, search_window=7, h=0.1):
        super(NLM_Model, self).__init__()
        self.patch_size = patch_size
        self.search_window = search_window
        self.h = h

    def forward(self, noisy_image):
        """
        noisy_image: (C, H, W) 텐서, C는 채널, H와 W는 높이와 너비
        """
        C, H, W = noisy_image.shape
        denoised_image = torch.zeros_like(noisy_image)
        
        # 패치 크기와 검색 창 크기 설정
        half_patch = self.patch_size // 2
        half_window = self.search_window // 2
        
        # Non-Local Means 알고리즘 적용
        for i in range(H):
            for j in range(W):
                # 현재 픽셀과 주변 패치를 비교
                patch_center = noisy_image[:, i, j]
                weight_sum = 0
                weighted_sum = torch.zeros(C)
                
                # 검색 창 내에서 유사한 패치 찾아 가중 평균 계산
                for x in range(max(0, i - half_window), min(H, i + half_window + 1)):
                    for y in range(max(0, j - half_window), min(W, j + half_window + 1)):
                        # 기준 패치와 주변 패치
                        patch_1 = noisy_image[:, max(0, i-half_patch):min(H, i+half_patch+1), max(0, j-half_patch):min(W, j+half_patch+1)]
                        patch_2 = noisy_image[:, max(0, x-half_patch):min(H, x+half_patch+1), max(0, y-half_patch):min(W, y+half_patch+1)]
                        
                        # 패치 간의 차이 계산
                        dist = torch.sum((patch_1 - patch_2) ** 2)
                        weight = torch.exp(-dist / (self.h ** 2))
                        
                        # 가중 평균
                        weighted_sum += weight * noisy_image[:, x, y]
                        weight_sum += weight
                
                # 현재 픽셀의 복원 값 설정
                denoised_image[:, i, j] = weighted_sum / weight_sum if weight_sum != 0 else patch_center
        
        return denoised_image



#### 3.6 GAN

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1)
        self.relu = nn.ReLU(True)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1)
        self.deconv1 = nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1)
        self.deconv2 = nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1)
        self.deconv3 = nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1)
        self.tanh = nn.Tanh()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.relu(self.deconv1(x))
        x = self.relu(self.deconv2(x))
        x = self.tanh(self.deconv3(x))
        return x
    
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1)
        self.fc = nn.Linear(256 * 4 * 4, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.LeakyReLU(self.conv1(x))
        x = self.LeakyReLU(self.conv2(x))
        x = self.LeakyReLU(self.conv3(x))
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.sigmoid(self.fc(x))
        return x


#### 3.7  DNCNN

### 4. 신경망 학습과 테스트 진행

#### Generic Train and evaluate function

In [ ]:
# Train and evaluate function
def train_and_evaluate(model, train_loader, test_loader, device = device, epochs=defualt_epoch, \
                       Data_Reset = Data_Reset, net_reset = False, folder_path = default_path, \
                       net_name = "defual_net.pt", log_save = log_save, log_name = "default_log"):
  
  if os.path.exists(folder_path)== False:
    print("디렉토리 없음")
    if net_reset == False and Data_Reset == False and Auto_Init == False:
      return
  
    os.mkdir(folder_path)
    print("디렉토리를 만들었습니다 : ", folder_path)
    gitignore(directory = folder_path)
  
  os.chdir(folder_path)

  if os.path.exists(net_name) == False and Auto_Init == False:
      print("학습데이터 없음")
      os.chdir("../")
      return

  if net_reset == True or Data_Reset == True or (os.path.exists(net_name) == False and Auto_Init == True):

    model.train()

    criterion = nn.CrossEntropyLoss()
    # optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    optimizer = optim.SGD(model.parameters(), lr=0.03, momentum=0.9, weight_decay=1e-4)

    accuracies = []

    # DataLoader에서 데이터 항목 개수 확인
    for batch in train_loader:
        num_items = len(batch)  # 배치에서 항목 개수 확인
        print("Number of Data Type:", num_items)
        break

    for epoch in range(epochs):
      running_loss = 0.0

      if num_items == 2:
        for data in train_loader:
            original_images, original_labels = data
            original_images, original_labels = original_images.to(device), original_labels.to(device)

            optimizer.zero_grad()

            outputs = model(original_images)
            loss = criterion(outputs, original_labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
      
      elif num_items == 4:
        for data in train_loader:
            _, _, poisoned_images, poisoned_labels = data
            poisoned_images, poisoned_labels = poisoned_images.to(device), poisoned_labels.to(device)

            optimizer.zero_grad()

            # 모델 예측
            outputs = model(poisoned_images)

            # 손실 계산
            loss = criterion(outputs, poisoned_labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
      elif num_items == 5:
        for data in train_loader:
            _, _, _, poisoned_labels, restored_images = data
            restored_images, poisoned_labels = restored_images.to(device), poisoned_labels.to(device)

            optimizer.zero_grad()

            # 모델 예측
            outputs = model(restored_images)

            # 손실 계산
            loss = criterion(outputs, poisoned_labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
      elif num_items == 6:
        for data in train_loader:
            _, _, _, poisoned_labels, _,  restored_images = data
            restored_images, poisoned_labels = restored_images.to(device), poisoned_labels.to(device)

            optimizer.zero_grad()

            # 모델 예측
            outputs = model(restored_images)

            # 손실 계산
            loss = criterion(outputs, poisoned_labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
      else :
          print("dataset error")
          os.chdir("../")
          return 
        
      print(f'[Epoch {epoch + 1}] loss: {running_loss / len(train_loader):.3f}')

      accuracy = evaluate(model, device, test_loader)
      accuracies.append(accuracy)
      print(f'Accuracy: {accuracy:.2f}%')

    torch.save(model, net_name)
    print("모델을 저장하였습니다 : ", net_name)
    
    if log_save == True or Data_Reset == True or (os.path.exists(net_name) == False and Auto_Init == True):
      np.save(log_name, accuracies)
      print("로그를 저장하였습니다 : ", log_name, ".npy")
    
    del model
    torch.cuda.empty_cache()
    gc.collect()
      
  
  else:  
    if os.path.exists(log_name+".npy")== False:
      print("accuracies 없음 (모델 로드가능)")
      accuracies = []

    else:  
      accuracies_np = np.load(log_name+".npy")
      accuracies = accuracies_np.tolist()
      print("accuracies 로드됨")
  print(os.getcwd())
  
  os.chdir("../")
  return accuracies

#### 4.1 Resnet

In [ ]:
# Evaluate function
def evaluate(model, device, test_loader, ex_load = False, load_dict = "./default_net.pt"):
  if ex_load == True :
    model = torch.load(load_dict)
  model.eval()
  total_correct = 0
  with torch.no_grad():
     for data, target in test_loader:
      data, target = data.to(device), target.to(device)
      output = model(data)
      _, preds = torch.max(output, 1)
      total_correct += (preds == target).sum().item()

  accuracy = 100 * total_correct / len(test_loader.dataset)
  return accuracy

#### 4.2 Image Restoration Neural Network Train Function
##### Autoencoder/ Unet

In [ ]:
# 학습 함수
def train_model(model, trainloader, device = device,  epochs=defualt_epoch, \
                       Data_Reset = Data_Reset, net_reset = False, folder_path = default_path, \
                       net_name = "defual_net.pt", log_save = log_save, log_name = "default_log"):
    
    if os.path.exists(folder_path)== False:
        print("디렉토리 없음")
        if net_reset == False and Data_Reset == False and Auto_Init == False:
            return
        
        os.mkdir(folder_path)
        print("디렉토리를 만들었습니다 : ", folder_path)
        gitignore(directory = folder_path)

    os.chdir(folder_path)

    if os.path.exists(net_name)== False and Auto_Init == False:
            print("학습데이터 없음")
            os.chdir("../")
            return
    
    # DataLoader에서 데이터 항목 개수 확인
    for batch in trainloader:
        num_items = len(batch)  # 배치에서 항목 개수 확인
        print("Number of Data Type:", num_items)
        break
    
    if net_reset == True or Data_Reset == True or (os.path.exists(net_name) == False and Auto_Init == True):
    
        model.train()
        loss = []

        criterion = nn.MSELoss()  # 평균 제곱 오차 (복원된 이미지와 원본 이미지 간의 차이)
        optimizer = optim.Adam(model.parameters(), lr=0.001)

        for epoch in range(epochs):
            running_loss = 0.0
            for data in trainloader:
                if num_items == 2 :
                    inputs, _ = data
                    inputs = inputs.to(device)

                    # 손상된 이미지 생성 (노이즈 추가)
                    poisoned = inputs + 0.1 * torch.randn_like(inputs)
                    poisoned = torch.clip(poisoned, 0., 1.)  # 값이 0~1 사이로 유지되도록 조정

                elif num_items == 4 :
                    inputs, _, poisoned, _ = data  # 레이블은 필요 없음
                    inputs = inputs.to(device)
                    poisoned = poisoned.to(device)

                optimizer.zero_grad()

                # 모델에 손상된 이미지 입력
                outputs = model(poisoned)
                
                # 손실 계산
                loss = criterion(outputs, inputs)  # 원본 이미지와 복원된 이미지 간의 차이
                loss.backward()
                optimizer.step()

                running_loss += loss.item()

            print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(trainloader)}')
        

        torch.save(model, net_name)
        print("모델을 저장하였습니다 : ", net_name)
        
        if log_save == True or Data_Reset == True or (os.path.exists(net_name) == False and Auto_Init == True):
            loss = loss.cpu()
            np.save(log_name, loss.detach().numpy())
            print("로그를 저장하였습니다 : ", log_name, ".npy")
        
        print(os.getcwd())    
        os.chdir("../")
        del model
        torch.cuda.empty_cache()
        gc.collect()
        
        return            
        
    else:      
        if os.path.exists(log_name+".npy")== False:
            print("accuracies 없음 (모델 로드 가능)")
            loss = []

        else:  
            loss = torch.from_numpy(np.load(log_name+".npy"))
            print("loss 로드됨")
        
        print(os.getcwd())
        os.chdir("../")
        return

#### 4.3 GAN

In [ ]:
# 학습 함수
def GAN_trainer(model_generator, model_discriminator, trainloader, device = device,  epochs=defualt_epoch, \
                       Data_Reset = Data_Reset, net_reset = False, folder_path = default_path, \
                       net_g_name = "defualt_net_g.pt", net_d_name = "defualt_net_d.pt"):
    
    if os.path.exists(folder_path)== False:
        print("디렉토리 없음")
        if net_reset == False and Data_Reset == False and Auto_Init == False:
            return
        
        os.mkdir(folder_path)
        print("디렉토리를 만들었습니다 : ", folder_path)
        gitignore(directory = folder_path)

    os.chdir(folder_path)

    if (not(os.path.exists(net_g_name)== True and os.path.exists(net_d_name)== True)) and Auto_Init == False:
            print("학습데이터 없음")
            os.chdir("../")
            return
    
    # DataLoader에서 데이터 항목 개수 확인
    for batch in trainloader:
        num_items = len(batch)  # 배치에서 항목 개수 확인
        print("Number of Data Type:", num_items)
        break
    
    if net_reset == True or Data_Reset == True or ((not(os.path.exists(net_g_name)== True and os.path.exists(net_d_name)== True)) and Auto_Init == True):
        
        # 손실 함수와 옵티마이저 정의
        criterion = nn.BCELoss()
        optimizer_g = optim.Adam(model_generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
        optimizer_d = optim.Adam(model_discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

        for epoch in range(epochs):
            for data in trainloader:
                if num_items == 2 :
                    inputs, _ = data
                    inputs = inputs.to(device)

                    # 손상된 이미지 생성 (노이즈 추가)
                    poisoned = inputs + 0.1 * torch.randn_like(inputs)
                    poisoned = torch.clip(poisoned, 0., 1.)  # 값이 0~1 사이로 유지되도록 조정

                elif num_items == 4 :
                    inputs, _, poisoned, _ = data  # 레이블은 필요 없음
                    inputs = inputs.to(device)
                    poisoned = poisoned.to(device)

                # 실제 레이블은 1, 가짜 레이블은 0
                real_labels = torch.ones(inputs.size(0), 1).cuda()
                fake_labels = torch.zeros(inputs.size(0), 1).cuda()

                # -----------------
                # Discriminator 훈련
                # -----------------
                optimizer_d.zero_grad()

                # 실제 이미지에 대해 Discriminator 훈련
                real_outputs = model_discriminator(inputs)
                d_loss_real = criterion(real_outputs, real_labels)

                # 가짜 이미지에 대해 Discriminator 훈련
                fake_imgs = model_generator(poisoned)
                fake_outputs = model_discriminator(fake_imgs.detach())
                d_loss_fake = criterion(fake_outputs, fake_labels)

                # 총 Discriminator 손실
                d_loss = d_loss_real + d_loss_fake
                d_loss.backward()
                optimizer_d.step()

                # -----------------
                # Generator 훈련
                # -----------------
                optimizer_g.zero_grad()

                # Generator는 Discriminator를 속이도록 훈련
                g_loss = criterion(model_discriminator(fake_imgs), real_labels)
                g_loss.backward()
                optimizer_g.step()

            print(f'Epoch [{epoch+1}/{epochs}], D Loss: {d_loss.item()}, G Loss: {g_loss.item()}')
        

        torch.save(model_discriminator, net_d_name)
        torch.save(model_generator, net_g_name)
        print("모델을 저장하였습니다 : ", net_g_name, net_d_name)
        

        
        print(os.getcwd())    
        os.chdir("../")
        del model_discriminator, model_generator
        torch.cuda.empty_cache()
        gc.collect()
        
        return            
        
    else:      
        print("(모델 로드 가능)")
        os.chdir("../")
        return


### 5. 데이터의 오염 통합 모델

In [ ]:
class AugmentedDataset(Dataset):
    """
    CIFAR-10 데이터셋에 다양한 공격을 추가하는 클래스입니다.
    """
    def __init__(self, dataset, perterbationset, attacked_ratio=0.2, label_attack=True, label_perterbation=True, \
                 overlay=True, alpha=0.5, partial_overlay=True, rotational_overlay=True, image_corruption=True, \
                    corruption_ratio=0.2, attack_order="label", im_proc_order="overlay"):
        """ AugmentedDataset 객체를 초기화합니다. """
        self.dataset = dataset
        self.perterbationset = perterbationset
        self.attacked_ratio = attacked_ratio
        self.label_attack = label_attack
        self.label_perterbation = label_perterbation
        self.overlay = overlay
        self.alpha = alpha
        self.partial_overlay = partial_overlay
        self.rotational_overlay = rotational_overlay
        self.image_corruption = image_corruption
        self.corruption_ratio = corruption_ratio
        self.attack_order = attack_order  # 공격 순서 설정
        self.im_proc_order = im_proc_order

        # 공격 인덱스 설정
        self.attacked_indices = self._select_attacked_indices(self.attacked_ratio)

        # Label 오염 생성 (라벨 공격이 활성화된 경우에만)
        if self.label_attack:
            self.attacked_labels = self._crazy_labels()
        else:
            self.attacked_labels = [label for _, label in self.dataset]  # 라벨 공격이 없으면 원본 라벨 사용

        # Label flipping을 위한 라벨 생성 (라벨 섭동이 활성화된 경우에만)
        if self.label_perterbation:
            self.perbutated_labels = self._label_flipper(self.attacked_labels)
        else:
            self.perbutated_labels = self.attacked_labels  # 라벨 섭동이 없으면 오염된 라벨을 그대로 사용

        # 변형된 이미지를 저장할 리스트 초기화
        self.transformed_images = [None] * len(self.dataset)

        # 이미지 오염 및 이미지 겹침 초기화 (이미지를 한 번만 오염 처리)
        self.processed_images = [None] * len(self.dataset)
        for idx in range(len(self.dataset)):
            image, _ = self.dataset[idx]
            image = self.__image_process__(image, idx)  # 초기화 시 한 번만 이미지 처리
            self.processed_images[idx] = image  # 처리된 이미지 저장

    def _select_attacked_indices(self, ratio):
        """공격 대상 샘플의 인덱스를 선택합니다."""
        num_attack_samples = int(ratio * len(self.dataset))
        return random.sample(range(len(self.dataset)), num_attack_samples)

    def _crazy_labels(self):
        """Label 오염된 라벨 리스트를 생성합니다."""
        attacked_labels = []
        for idx in range(len(self.dataset)):
            _, original_label = self.dataset[idx]
            if idx in self.attacked_indices:
                # 원래 라벨과 다른 무작위 라벨 생성
                attacked_label = original_label
                while attacked_label == original_label:
                    attacked_label = random.randint(0, 9)
                attacked_labels.append(attacked_label)
            else:
                attacked_labels.append(original_label)
        return attacked_labels

    def _label_flipper(self, target):
        """
        CIFAR-10 데이터셋의 일부 샘플에 Label-Flipping 오염을 가하는 함수입니다.
        """
        perbutated_labels = []

        for idx in range(len(target)):
            label = target[idx]

            if idx in self.attacked_indices:
                # 라벨 복사 후 공격 수행
                original_label = label

                if original_label == 1:
                    label = 9
                elif original_label == 9:
                    label = 1
                elif original_label == 5:
                    label = 3
                elif original_label == 3:
                    label = 5
                elif original_label == 7:
                    label = 4
                elif original_label == 4:
                    label = 7
                elif original_label == 2:
                    label = 6
                elif original_label == 6:
                    label = 2
                elif original_label == 0:
                    label = 8
                elif original_label == 8:
                    label = 0

            perbutated_labels.append(label)

        return perbutated_labels

    def __len__(self):
        """데이터셋의 전체 길이를 반환합니다."""
        return len(self.dataset)

    def __image_process__(self, image, idx):
        """이미지 처리 순서 함수"""
        if self.im_proc_order == "overlay":
            image = self._overlay(image, idx) if self.overlay == True else image
            image = self._corrupt_image(image, idx) if self.image_corruption == True else image
        elif self.im_proc_order == "corrupt":
            image = self._corrupt_image(image, idx) if self.image_corruption == True else image
            image = self._overlay(image, idx) if self.overlay == True else image
        else:
            raise ValueError(f"Invalid im_proc_order: {self.im_proc_order}. Use 'overlay' or 'corrupt'.")

        return image

    def _overlay(self, image, idx):
        """특정 이미지에 이미지 겹침 공격을 적용합니다."""
        if idx in self.attacked_indices:
            if self.partial_overlay:
                # 텐서를 PIL 이미지로 변환
                pil_image = transforms.ToPILImage()(image)

                # 두 개의 이미지를 랜덤으로 선택하여 중첩
                rand_idx_1 = random.randint(0, len(self.dataset) - 1)
                rand_idx_2 = random.randint(0, len(self.dataset) - 1)

                overlay_image_1, _ = self.dataset[rand_idx_1]
                overlay_image_2, _ = self.dataset[rand_idx_2]

                # 첫 번째 이미지를 중첩
                overlay_h_1, overlay_w_1 = overlay_image_1.shape[1], overlay_image_1.shape[2]
                scale_factor_1 = random.uniform(0.5, 1.0)
                new_h_1 = int(overlay_h_1 * scale_factor_1)
                new_w_1 = int(overlay_w_1 * scale_factor_1)

                overlay_image_1_pil = transforms.ToPILImage()(overlay_image_1)
                overlay_image_1_resized = overlay_image_1_pil.resize((new_w_1, new_h_1))

                x1 = random.randint(0, pil_image.size[0] - new_w_1)
                y1 = random.randint(0, pil_image.size[1] - new_h_1)
                pil_image.paste(overlay_image_1_resized, (x1, y1))

                # 두 번째 이미지를 중첩
                overlay_h_2, overlay_w_2 = overlay_image_2.shape[1], overlay_image_2.shape[2]
                scale_factor_2 = random.uniform(0.5, 1.0)
                new_h_2 = int(overlay_h_2 * scale_factor_2)
                new_w_2 = int(overlay_w_2 * scale_factor_2)

                overlay_image_2_pil = transforms.ToPILImage()(overlay_image_2)
                overlay_image_2_resized = overlay_image_2_pil.resize((new_w_2, new_h_2))

                x2 = random.randint(0, pil_image.size[0] - new_w_2)
                y2 = random.randint(0, pil_image.size[1] - new_h_2)
                pil_image.paste(overlay_image_2_resized, (x2, y2))

                # 최종 이미지를 텐서로 변환하여 저장
                final_image = transforms.ToTensor()(pil_image)
                return final_image
        
        return image  # 이미지가 공격되지 않으면 원본 이미지 그대로 반환

    def _corrupt_image(self, image, idx):
        """이미지의 일부 픽셀을 결손 처리합니다."""
        if self.image_corruption:
            if idx in self.attacked_indices:
                image_np = image.numpy()  # 텐서를 NumPy 배열로 변환
                total_pixels = image_np.size
                num_corrupted_pixels = int(total_pixels * self.corruption_ratio)

                # 랜덤으로 픽셀 선택 (이미지의 플랫(flat) 배열에서 인덱스 기준)
                indices = random.sample(range(total_pixels), num_corrupted_pixels)
                flat_image = image_np.flatten()

                # 선택된 픽셀을 0으로 설정
                flat_image[indices] = 0

                # 이미지를 원래 형태로 복원
                image_np = flat_image.reshape(image_np.shape)
                return torch.tensor(image_np)

        return image

    def __getitem__(self, idx):
        """
        데이터셋의 특정 샘플을 반환합니다.
        """
        # 원본 이미지와 라벨 가져오기
        original_images, original_label = self.dataset[idx]

        # 라벨 오염 및 이미지 오염 처리된 데이터 반환
        image = self.processed_images[idx]
        label = self.perbutated_labels[idx]

        return original_images, original_label, image, label


In [ ]:
## Data Save 함수
def save_dataset (data, folder_path = default_path, dat_name = "defual_net.dat"):
    if os.path.exists(folder_path)== False:
        print("디렉토리 없음")
    
        os.mkdir(folder_path)
        print("디렉토리를 만들었습니다 : ", folder_path)
        gitignore(directory = folder_path)
  
    os.chdir(folder_path)
    torch.save(data, dat_name)
    print("data saved")
    os.chdir("../")

    torch.cuda.empty_cache()
    gc.collect()
    return

## Data Load 함수
def load_dataset (folder_path = default_path, dat_name = "defual_net.dat"):
    if os.path.exists(folder_path)== False:
        print("디렉토리 없음")
        return
    
    os.chdir(folder_path)
    if os.path.exists(dat_name) == False:
        print("데이터 없음")
        os.chdir("..//")
        return
    
    data = torch.load(dat_name)
    os.chdir("..//")
    return data
  

### 실험 진행

##### 1. 오염 없는 일반적 데이터 학습 과정

In [ ]:
# ResNet50 모델 신경망 생성 및 장치에 저장
model_res = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=10).to(device)

In [ ]:
acc_res = train_and_evaluate(model_res, train_loader, test_loader, net_reset=poision_sys_reset, net_name="default_resnet.pt", log_name = "default_log")

##### 2. 오염을 적용하는 데이터 학습 과정

In [ ]:
train_set_attacked = load_dataset(folder_path = default_path, dat_name = "poisoned.dat")
if train_set_attacked == None or Data_Reset == True or poision_sys_reset == True:
    train_set_attacked = AugmentedDataset(
        dataset=train_set,
        perterbationset=add_set,
        attacked_ratio=0.4,
        overlay=False,
        alpha=0.25,
        label_attack=True,
        partial_overlay=True,
        corruption_ratio=0.2,
        image_corruption=True,
        attack_order="label",
        im_proc_order="overlay"
    )
    save_dataset(train_set_attacked, dat_name = "poisoned.dat")

train_loader_attacked = DataLoader(dataset=train_set_attacked, batch_size=64, shuffle=True)

del train_set_attacked

In [ ]:
for batch in train_loader_attacked:
    original_images, original_labels, poisoned_images, poisoned_labels = batch
    break  # 모든 데이터를 한 번에 가져왔으므로 반복문 종료

# 확인
print(original_images.shape)
print(original_labels.shape)
print(poisoned_images.shape)
print(poisoned_labels.shape)

In [ ]:
visualize_multiple_dataloaders([train_loader, train_loader_attacked, train_loader_attacked, train_loader_attacked, train_loader_attacked], test_loader, images_per_loader=5)

In [ ]:
# ResNet50 모델 신경망 생성 및 장치에 저장
model_res_1 = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=10).to(device)

In [ ]:
acc_res_1 = train_and_evaluate(model_res_1, train_loader_attacked, test_loader, net_reset=poision_sys_reset, net_name="train_loader_attacked.pt", log_name = "train_loader_attacked")

### 최종 비교 그래프

In [ ]:
# Visualize the test accuracy
plt.figure(figsize=(10, 5))

plt.plot(acc_res, marker='o', linestyle='-', markersize=5, label='ResNet_Default')
plt.plot(acc_res_1, marker='o', linestyle='-', markersize=5, label='ResNet_op')

plt.title('Model Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')

plt.grid(True)
plt.legend()
plt.show()

In [ ]:
visualize_multiple_dataloaders([train_loader, train_loader_attacked, train_loader_attacked, train_loader_attacked, train_loader_attacked], test_loader, images_per_loader=5)

#### Poison Activation Validator

In [ ]:
train_set_attacked_validate = load_dataset(folder_path = default_path, dat_name = "poisoned_validate.dat")
if train_set_attacked_validate== None or Data_Reset == True or poision_sys_reset == True:
    train_set_attacked_validate = AugmentedDataset(
        dataset=train_set,
        perterbationset=add_set,
        attacked_ratio=1,
        overlay=False,
        alpha=0.25,
        label_attack=True,
        partial_overlay=True,
        image_corruption= True,
        corruption_ratio=0.2,
        attack_order="label",
        im_proc_order="overlay"
    )
    save_dataset(train_set_attacked_validate, dat_name = "poisoned_validate.dat")

train_loader_attacked_validate = DataLoader(dataset=train_set_attacked_validate, batch_size=64, shuffle=True)

del train_loader_attacked_validate
del train_set_attacked_validate

### 6. Image Restoration Module

#### 6.1 중독데이터 복원 AI 학습

##### CIFAR-10을 학습데이터로 이용

In [ ]:
auto_10 = Autoencoder().to(device)
unet_10 = UNet().to(device)
tsme_10 = TSNE_Model().to(device)
nlm_10 = NLM_Model().to(device)
# GAN 모델 초기화
Gan_g_10 = Generator().cuda()
Gan_d_10 = Discriminator().cuda()

Autotest = train_model(auto_10, train_loader, device = device, net_reset= restore_sys_reset, net_name="auto_10.pt", log_name = "auto_10")
unettest = train_model(unet_10, train_loader, device = device, net_reset= restore_sys_reset, net_name="unet_10.pt", log_name = "unet_10")
#tsmetest = train_model(tsme_10, train_loader, device = device, net_reset= restore_sys_reset, net_name="tsme_10.pt", log_name = "tsme_10")
#nlmtest = train_model(nlm_10, train_loader, device = device, net_reset= restore_sys_reset, net_name="nlm_10.pt", log_name = "nlm_10")
GANtest = GAN_trainer(Gan_g_10, Gan_d_10, train_loader, device = device, net_reset= restore_sys_reset, net_name="unet_10.pt")



##### 중독데이터를 학습데이터로 이용

In [ ]:
auto_attacked = Autoencoder().to(device)
unet_attacked = UNet().to(device)

Autotest_attaked = train_model(auto_attacked, train_loader_attacked, net_reset= restore_sys_reset, net_name="auto_attacked.pt", log_name = "auto_attacked")
unettest_attaked = train_model(unet_attacked, train_loader_attacked, net_reset= restore_sys_reset, net_name="unet_attacked.pt", log_name = "unet_attacked")

##### CIFAR-100을 학습데이터로 이용

In [ ]:
auto_100 = Autoencoder().to(device)
unet_100 = UNet().to(device)

Autotest_100 = train_model(auto_100, add_loader, net_reset= restore_sys_reset, net_name="auto_100.pt", log_name = "auto_100")
unettest_100 = train_model(unet_100, add_loader, net_reset= restore_sys_reset, net_name="unet_100.pt", log_name = "unet_100")

#### 6.2 학습결과 표출

In [ ]:
# 이미지 결과 확인 함수
def show_images(restored_data, index = 0):
    # 결과 이미지를 시각화
    if isinstance(restored_data, restored_Items):
        o_image, o_label, p_image, p_label, r_cond, r_image = restored_data.original_images, restored_data.original_labels, restored_data.poisoned_images, restored_data.poisoned_labels, restored_data.condition_code, restored_data.restored_images
    else :
        o_image, o_label, p_image, p_label, r_image = restored_data.original_images, restored_data.original_labels, restored_data.poisoned_images, restored_data.poisoned_labels, restored_data.restored_images
    fig, ax = plt.subplots(1, 3, figsize=(12, 4))
    
    ax[0].imshow(o_image[index].permute(1, 2, 0).cpu().detach().numpy())
    ax[0].set_title('Original')
    ax[1].imshow(p_image[index].permute(1, 2, 0).cpu().detach().numpy())
    ax[1].set_title('Poisoned')
    ax[2].imshow(r_image[index].permute(1, 2, 0).cpu().detach().numpy())
    ax[2].set_title('Recovered Output')
    
    plt.show()
    print("Original Label:", o_label[index])
    print("Poisoned Label:", p_label[index])
    if isinstance(restored_data, restored_Items):
        print("Poisoned condition:", r_cond[index])
    del restored_data
    torch.cuda.empty_cache()
    gc.collect()

def multi_show(restored_data, sample = 10):
    for i in range(sample):
        label = restored_data.poisoned_labels
        index = random.randrange(1, len(label))
        show_images(restored_data, index)
    del restored_data
    torch.cuda.empty_cache()
    gc.collect()

#### 6.3 중독 데이터 분류

##### 블러감지

In [ ]:
def blur_IO(image_in, threshold=0.5):
    # 이미지 읽기
    image_b = image_in.cpu().numpy().transpose((1, 2, 0))  # (C, H, W) -> (H, W, C)

    # PyTorch는 RGB 순서이므로 BGR로 변환해야 함
    image_b = cv2.cvtColor(image_b, cv2.COLOR_RGB2BGR)
    
    # 이미지를 그레이스케일로 변환
    gray_image = cv2.cvtColor(image_b, cv2.COLOR_BGR2GRAY)
    
    # Laplacian을 사용하여 이미지의 엣지 강도를 계산
    variance = cv2.Laplacian(gray_image, cv2.CV_32F).var()
    
    # 분산 값이 낮으면 블러링이 발생한 것으로 판단
    if variance < threshold:
        return True  # 이미지가 블러됨
    else:
        return False  # 이미지가 선명함


##### 노이즈 감지

In [ ]:
def noise_IO(image_in, threshold=5.0):
    # 이미지 읽기
    image_n = image_in.cpu().numpy().transpose((1, 2, 0))  # (C, H, W) -> (H, W, C)
    # PyTorch는 RGB 순서이므로 BGR로 변환해야 함
    image_n = cv2.cvtColor(image_n, cv2.COLOR_RGB2BGR)
    
    # 이미지를 그레이스케일로 변환
    gray_image = cv2.cvtColor(image_n, cv2.COLOR_BGR2GRAY)
    
    # 이미지의 표준 편차를 계산
    std_dev = np.std(gray_image)
    
    # 표준 편차가 일정 값 이상이면 노이즈가 있다고 판단
    if std_dev > threshold:
        return True  # 이미지에 노이즈가 있음
    else:
        return False  # 이미지에 노이즈 없음


##### 변형감지

In [ ]:
def c_distortion_IO(image_in, threshold=140):                ##Colour Distortion
    # 이미지 읽기
    image_d = image_in.cpu().numpy().transpose((1, 2, 0))  # (C, H, W) -> (H, W, C)
    # PyTorch는 RGB 순서이므로 BGR로 변환해야 함
    image_d = cv2.cvtColor(image_d, cv2.COLOR_RGB2BGR)
    
    # 이미지를 BGR에서 HSV로 변환
    hsv_image = cv2.cvtColor(image_d, cv2.COLOR_BGR2HSV)
    
    # HSV 채널 분리
    h, s, v = cv2.split(hsv_image)
    
    # 색상의 평균값과 표준 편차 계산
    mean_hue = np.mean(h)
    mean_saturation = np.mean(s)
    mean_value = np.mean(v)
    
    # 왜곡이 있을 경우 평균값이 비정상적으로 클 수 있음
    if mean_hue > threshold or mean_saturation > threshold or mean_value > threshold:
        return True  # 색 왜곡 있음
    else:
        return False  # 색 왜곡 없음


#### 6.4 중독된 데이터 복구 적용

#### 복구데이터 클래스 정의

In [ ]:
class CIFAR10WithModelOutputAndLabels(Dataset):
    def __init__(self, original_images, original_labels, poisoned_images,  poisoned_labels, restored_images):
        self.original_images= original_images
        self.original_labels = original_labels        
        self.poisoned_images = poisoned_images
        self.poisoned_labels = poisoned_labels
        self.restored_images = restored_images

    def __len__(self):
        return len(self.poisoned_images)

    def __getitem__(self, idx):
        return self.original_images[idx], self.original_labels[idx], self.poisoned_images[idx], self.poisoned_labels[idx] , self.restored_images[idx]

# 이미지를 모델에 통과시키고 출력 저장하는 함수
def process_and_save_cifar10(dataloader, net_name ="default_net.pt" , output_filename="processed_cifar10_with_labels.pth"):
    """
    CIFAR-10 DataLoader에서 이미지를 처리하고, 모델의 출력과 라벨을 저장하는 함수.
    """
    os.chdir(default_path)
    if (os.path.exists(output_filename) == False and Auto_Init == True) or Data_Reset == True:
        model = torch.load(net_name)
        model.eval()  # 모델을 평가 모드로 설정
        os.chdir("../")

        all_original_images =[]
        all_original_labels=[]
        all_images = []
        all_outputs = []
        all_labels = []

        with torch.no_grad():  # 예측 시에는 그래디언트 계산을 하지 않음
            for data in dataloader:
                original_images, original_labels, images, labels = data
                # 이미지를 GPU로 이동 (GPU가 사용 가능하면)
                images = images.cuda() if torch.cuda.is_available() else images
                labels = labels.cuda() if torch.cuda.is_available() else labels

                # 모델을 통해 예측 결과 얻기
                outputs = model(images)

                # 결과를 리스트에 추가
                all_original_images.append(original_images)
                all_original_labels.append(original_labels)
                all_images.append(images.cpu())  # CPU로 이동하여 저장
                all_labels.append(labels.cpu())  # CPU로 이동하여 저장
                all_outputs.append(outputs.cpu())  # CPU로 이동하여 저장

        # CIFAR-10 이미지, 모델 출력, 라벨을 포함한 Dataset 생성
        combined_dataset = CIFAR10WithModelOutputAndLabels(
            torch.cat(all_original_images), torch.cat(all_original_labels), torch.cat(all_images), torch.cat(all_labels) , torch.cat(all_outputs)
        )

        os.chdir(default_path)
        # PyTorch Dataset을 저장
        torch.save(combined_dataset, output_filename)
        print(f"Processed data with labels saved to {output_filename}")
        
        del model
        torch.cuda.empty_cache()
        gc.collect()
    
    elif Auto_Init == False:
        print("학습을 생략")
    else :
        print("학습된 파일이 있습니다")
    print(os.getcwd)
    os.chdir("../")



In [ ]:
class restored_Items(Dataset):
    " Restoration_Condition 추가"
    def __init__(self, original_images, original_labels, poisoned_images,  poisoned_labels, condition_code, restored_images):
        self.original_images= original_images
        self.original_labels = original_labels        
        self.poisoned_images = poisoned_images
        self.poisoned_labels = poisoned_labels
        self.condition_code = condition_code                ## 0: non poisoned, 1: blur, 2: noise, 4:destortion = condition
        self.restored_images = restored_images 

    def __len__(self):
        return len(self.poisoned_images)

    def __getitem__(self, idx):
        return self.original_images[idx], self.original_labels[idx], self.poisoned_images[idx], self.poisoned_labels[idx], self.condition_code[idx], self.restored_images[idx]
    
    def _code_(self, idx):
        if self.condition_code == 0 :
            code_name = " - "
        else : 
            code_name = "에러" + str(self.condition_code)
        return code_name

# 이미지를 모델에 통과시키고 출력 저장하는 함수
def restore(dataloader, restore_reset_IO = reclass_sys_reset, net_name ="default_net.pt" ,net2_name = None , output_filename="processed_cifar10_with_labels.pth"):
    """
    CIFAR-10 DataLoader에서 이미지를 처리하고, 모델의 출력과 라벨을 저장하는 함수.
    """
    os.chdir(default_path)
    if (os.path.exists(output_filename) == False and Auto_Init == True) or Data_Reset == True or restore_reset_IO == True:
        model = torch.load(net_name)
        if not net2_name == None:
             model2 = torch.load(net2_name)
             model2.eval()
        model.eval()  # 모델을 평가 모드로 설정
        os.chdir("../")

        all_original_images =[]
        all_original_labels=[]
        all_images = []
        all_outputs = []
        all_condition_code = []
        all_labels = []

        with torch.no_grad():  # 예측 시에는 그래디언트 계산을 하지 않음
            for data in dataloader:
                original_images, original_labels, images, labels = data               
                batch_condition_code = []
                all_original_images.append(original_images)
                all_original_labels.append(original_labels)
                all_images.append(images.cpu())  # CPU로 이동하여 저장
                all_labels.append(labels.cpu())  # CPU로 이동하여 저장

                for i in range(images.size(0)): 
                    condition_code = 0
                    if blur_IO(images[i]) == True:
                            condition_code = condition_code + 1
                    if noise_IO(images[i]) == True:
                            condition_code = condition_code + 2
                    if c_distortion_IO(images[i]) == True:
                            condition_code = condition_code + 4
                    batch_condition_code.append(condition_code)

                # 이미지를 GPU로 이동 (GPU가 사용 가능하면)
                images = images.cuda() if torch.cuda.is_available() else images
                po_images = images.cuda() if torch.cuda.is_available() else images
                temp_images = images.cuda() if torch.cuda.is_available() else images
                #labels = labels.cuda() if torch.cuda.is_available() else labels
                output1 = model(images)
                outputs = po_images
                if not net2_name == None:
                     output2  =model2(temp_images)
                     outputC = model2(output1)
                
                # 모델을 통해 예측 결과 얻기
                for i in range(images.size(0)):
                    if batch_condition_code[i] == 4 and not net2_name == None:
                       outputs[i] = output2[i] 
                    elif batch_condition_code[i] >4 and not net2_name == None:
                         outputs[i] = outputC[i]
                    elif batch_condition_code[i] > 0:
                         outputs[i] = output1[i]


                # 결과를 리스트에 추가

                all_condition_code.append(torch.tensor(batch_condition_code))
                all_outputs.append(outputs.cpu())  # CPU로 이동하여 저장
                

        # CIFAR-10 이미지, 모델 출력, 라벨을 포함한 Dataset 생성
        combined_dataset = restored_Items(
            torch.cat(all_original_images), torch.cat(all_original_labels), torch.cat(all_images), torch.cat(all_labels), torch.cat(all_condition_code), torch.cat(all_outputs)
        )

        os.chdir(default_path)
        # PyTorch Dataset을 저장
        torch.save(combined_dataset, output_filename)
        print(f"Processed data with labels saved to {output_filename}")
        
        del model
        torch.cuda.empty_cache()
        gc.collect()
    
    elif Auto_Init == False:
        print("학습을 생략")
    else :
        print("학습된 파일이 있습니다")
    print(os.getcwd)
    os.chdir("../")



#### 복구

In [ ]:
# 10
if __name__ == "__main__":
    # CIFAR-10 DataLoader 로드

    # 학습된 unet_10 모델을 사용하여 CIFAR-10 데이터 처리 후 저장
    restore(train_loader_attacked, net_name = "auto_10.pt", net2_name= "unet_10.pt", output_filename="combie_10.pth")

# 저장된 파일 로드 예시
os.chdir(default_path)
s_combie_10 = torch.load("combie_10.pth")
os.chdir("../")

# 데이터, 출력, 라벨 가져오기
multi_show(s_combie_10, sample = 10)

del s_combie_10
torch.cuda.empty_cache()
gc.collect()

In [ ]:
# attack
if __name__ == "__main__":
    # CIFAR-10 DataLoader 로드

    # 학습된 unet_10 모델을 사용하여 CIFAR-10 데이터 처리 후 저장
    restore(train_loader_attacked, net_name = "auto_attacked.pt", net2_name= "unet_attacked.pt", output_filename="combie_attacked.pth")

# 저장된 파일 로드 예시
os.chdir(default_path)
s_combie_10 = torch.load("combie_attacked.pth")
os.chdir("../")

# 데이터, 출력, 라벨 가져오기
multi_show(s_combie_10, sample = 10)

del s_combie_10
torch.cuda.empty_cache()
gc.collect()

In [ ]:
# 100
if __name__ == "__main__":
    # CIFAR-10 DataLoader 로드

    # 학습된 unet_10 모델을 사용하여 CIFAR-10 데이터 처리 후 저장
    restore(train_loader_attacked, net_name = "auto_100.pt", net2_name= "unet_100.pt", output_filename="combie_100.pth")

# 저장된 파일 로드 예시
os.chdir(default_path)
s_combie_100 = torch.load("combie_100.pth")
os.chdir("../")

# 데이터, 출력, 라벨 가져오기
multi_show(s_combie_100, sample = 10)

del s_combie_100
torch.cuda.empty_cache()
gc.collect()

### 7. 복원 이미지 분류

In [ ]:
# ResNet50 모델 신경망 생성 및 장치에 저장
model_combie_10 = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=10).to(device)

model_combie_attack = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=10).to(device)

model_combie_100 = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=10).to(device)

In [ ]:
os.chdir(default_path)
s_combie_10 = torch.load("combie_10.pth")
os.chdir("../")

train_loader_auto_10 = DataLoader(dataset=s_combie_10, batch_size=64, shuffle=True)

acc_auto_10 = train_and_evaluate(model_combie_10, train_loader_auto_10, test_loader, net_reset=reclass_sys_reset, net_name="auto_10_7.pt", log_name = "auto_10_7")

In [ ]:
os.chdir(default_path)
s_auto_attacked = torch.load("combie_attacked.pth")
os.chdir("../")

train_loader_auto_attack = DataLoader(dataset=s_auto_attacked, batch_size=64, shuffle=True)

acc_auto_attack = train_and_evaluate(model_combie_attack, train_loader_auto_attack, test_loader, net_reset=reclass_sys_reset, net_name="auto_attacked_7.pt", log_name = "auto_attacked_7")

In [ ]:
os.chdir(default_path)
s_auto_100 = torch.load("combie_100.pth")
os.chdir("../")

train_loader_auto_100 = DataLoader(dataset=s_auto_100, batch_size=64, shuffle=True)

acc_auto_100 = train_and_evaluate(model_combie_100, train_loader_auto_100, test_loader, net_reset=reclass_sys_reset, net_name="auto_100_7.pt", log_name = "auto_100_7")

In [ ]:
# Visualize the test accuracy
plt.figure(figsize=(10, 5))

plt.plot(acc_res, marker='o', linestyle='-', markersize=5, label='ResNet_Default')
plt.plot(acc_res_1, marker='o', linestyle='-', markersize=5, label='Poisoned')
plt.plot(acc_auto_10, marker='x', linestyle='-', markersize=5, label='auto_10')

plt.plot(acc_auto_attack, marker='x', linestyle='-', markersize=5, label='auto_attack')

plt.plot(acc_auto_100, marker='x', linestyle='-', markersize=5, label='auto_100')


plt.title('Model Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')

plt.grid(True)
plt.legend()
plt.show()

### 8. Classification Multiplied 

In [ ]:
# ResNet50 모델 신경망 생성 및 장치에 저장
model_auto_10_2x = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=20).to(device)
model_unet_10_2x = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=20).to(device)

model_auto_attacked_2x = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=20).to(device)
model_unet_attacked_2x = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=20).to(device)

model_auto_100_2x = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=20).to(device)
model_unet_100_2x = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=20).to(device)

In [ ]:
# ResNet50 모델 신경망 생성 및 장치에 저장
model_auto_10_3x = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=30).to(device)
model_unet_10_3x = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=30).to(device)

model_auto_attacked_3x = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=30).to(device)
model_unet_attacked_3x = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=30).to(device)

model_auto_100_3x = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=30).to(device)
model_unet_100_3x = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=30).to(device)

In [ ]:
# ResNet50 모델 신경망 생성 및 장치에 저장
model_auto_10_5x = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=50).to(device)
model_unet_10_5x = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=50).to(device)

model_auto_attacked_5x = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=50).to(device)
model_unet_attacked_5x = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=50).to(device)

model_auto_100_5x = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=50).to(device)
model_unet_100_5x = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=50).to(device)

In [ ]:
# ResNet50 모델 신경망 생성 및 장치에 저장
model_auto_10_10x = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=100).to(device)
model_unet_10_10x = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=100).to(device)

model_auto_attacked_10x = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=100).to(device)
model_unet_attacked_10x = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=100).to(device)

model_auto_100_10x = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=100).to(device)
model_unet_100_10x = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=100).to(device)

#### Auto_10

In [ ]:
train_loader_auto_10_2x = DataLoader(dataset=s_auto_10, batch_size=64, shuffle=True)

acc_auto_10_2x = train_and_evaluate(model_auto_10_2x, train_loader_auto_10_2x, test_loader, net_reset=(multiplier_sys_reset or M_reset_auto_10), net_name="auto_10_2x.pt", log_name = "auto_10_2x")

In [ ]:
train_loader_auto_10_3x = DataLoader(dataset=s_auto_10, batch_size=64, shuffle=True)

acc_auto_10_3x = train_and_evaluate(model_auto_10_3x, train_loader_auto_10_3x, test_loader, net_reset=(multiplier_sys_reset or M_reset_auto_10), net_name="auto_10.3x", log_name = "auto_10_3x")

In [ ]:
train_loader_auto_10_5x = DataLoader(dataset=s_auto_10, batch_size=64, shuffle=True)

acc_auto_10_5x = train_and_evaluate(model_auto_10_5x, train_loader_auto_10_5x, test_loader, net_reset=(multiplier_sys_reset or M_reset_auto_10), net_name="auto_10_5x.pt", log_name = "auto_10_5x")

In [ ]:
train_loader_auto_10_10x = DataLoader(dataset=s_auto_10, batch_size=64, shuffle=True)

acc_auto_10_10x = train_and_evaluate(model_auto_10_10x, train_loader_auto_10_10x, test_loader, net_reset=(multiplier_sys_reset or M_reset_auto_10), net_name="auto_10_10x.pt", log_name = "auto_10_10x")

In [ ]:
# Visualize the test accuracy
plt.figure(figsize=(10, 5))

plt.plot(acc_res, marker='o', linestyle='-', markersize=5, label='ResNet_Default')
plt.plot(acc_res_1, marker='o', linestyle='-', markersize=5, label='Poisoned')
plt.plot(acc_auto_10, marker='x', linestyle='-', markersize=5, label='auto_10')
plt.plot(acc_auto_10_2x, marker='+', linestyle='-', markersize=5, label='auto_10_2x')
plt.plot(acc_auto_10_3x, marker='+', linestyle='-', markersize=5, label='auto_10_3x')
plt.plot(acc_auto_10_5x, marker='+', linestyle='-', markersize=5, label='auto_10_5x')
plt.plot(acc_auto_10_10x, marker='+', linestyle='-', markersize=5, label='auto_10_10x')
# plt.plot(acc_unet_10, marker='x', linestyle='-', markersize=5, label='unet_10')
# plt.plot(acc_auto_attack, marker='x', linestyle='-', markersize=5, label='auto_attack')
# plt.plot(acc_auto_attack, marker='x', linestyle='-', markersize=5, label='unet_attack')
# plt.plot(acc_auto_100, marker='x', linestyle='-', markersize=5, label='auto_100')
# plt.plot(acc_unet_100, marker='x', linestyle='-', markersize=5, label='unet_100')

plt.title('Model Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')

plt.grid(True)
plt.legend()
plt.show()

#### unet_10

In [ ]:
train_loader_unet_10_2x = DataLoader(dataset=s_unet_10, batch_size=64, shuffle=True)

acc_unet_10_2x = train_and_evaluate(model_unet_10_2x, train_loader_unet_10_2x, test_loader, net_reset=(multiplier_sys_reset or M_reset_unet_10), net_name="unet_10_2x.pt", log_name = "unet_10_2x")

In [ ]:
train_loader_unet_10_3x = DataLoader(dataset=s_unet_10, batch_size=64, shuffle=True)

acc_unet_10_3x = train_and_evaluate(model_unet_10_3x, train_loader_unet_10_3x, test_loader, net_reset=(multiplier_sys_reset or M_reset_unet_10), net_name="unet_10_3x.pt", log_name = "unet_10_3x")

In [ ]:
train_loader_unet_10_5x = DataLoader(dataset=s_unet_10, batch_size=64, shuffle=True)

acc_unet_10_5x = train_and_evaluate(model_unet_10_5x, train_loader_unet_10_5x, test_loader, net_reset=(multiplier_sys_reset or M_reset_unet_10), net_name="unet_10_5x.pt", log_name = "unet_10_5x")

In [ ]:
train_loader_unet_10_10x = DataLoader(dataset=s_unet_10, batch_size=64, shuffle=True)

acc_unet_10_10x = train_and_evaluate(model_unet_10_2x, train_loader_unet_10_10x, test_loader, net_reset=(multiplier_sys_reset or M_reset_unet_10), net_name="unet_10_10x.pt", log_name = "unet_10_10x")

In [ ]:
# Visualize the test accuracy
plt.figure(figsize=(10, 5))

plt.plot(acc_res, marker='o', linestyle='-', markersize=5, label='ResNet_Default')
plt.plot(acc_res_1, marker='o', linestyle='-', markersize=5, label='Poisoned')
plt.plot(acc_auto_10, marker='x', linestyle='-', markersize=5, label='auto_10')
plt.plot(acc_auto_10_2x, marker='+', linestyle='-', markersize=5, label='auto_10_2x')
plt.plot(acc_auto_10_3x, marker='+', linestyle='-', markersize=5, label='auto_10_3x')
plt.plot(acc_auto_10_5x, marker='+', linestyle='-', markersize=5, label='auto_10_5x')
plt.plot(acc_auto_10_10x, marker='+', linestyle='-', markersize=5, label='auto_10_10x')
# plt.plot(acc_unet_10, marker='x', linestyle='-', markersize=5, label='unet_10')
# plt.plot(acc_auto_attack, marker='x', linestyle='-', markersize=5, label='auto_attack')
# plt.plot(acc_auto_attack, marker='x', linestyle='-', markersize=5, label='unet_attack')
# plt.plot(acc_auto_100, marker='x', linestyle='-', markersize=5, label='auto_100')
# plt.plot(acc_unet_100, marker='x', linestyle='-', markersize=5, label='unet_100')

plt.title('Model Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')

plt.grid(True)
plt.legend()
plt.show()

#### auto_attacked

In [ ]:
train_loader_auto_attacked_2x = DataLoader(dataset=s_auto_attacked, batch_size=64, shuffle=True)

acc_auto_attacked_2x = train_and_evaluate(model_auto_attacked_2x, train_loader_auto_attacked_2x, test_loader, net_reset=(multiplier_sys_reset or M_reset_auto_attacked), net_name="auto_attacked_2x.pt", log_name = "auto_attacked_2x")

In [ ]:
train_loader_auto_attacked_3x = DataLoader(dataset=s_auto_attacked, batch_size=64, shuffle=True)

acc_auto_attacked_3x = train_and_evaluate(model_auto_attacked_3x, train_loader_auto_attacked_3x, test_loader, net_reset=(multiplier_sys_reset or M_reset_auto_attacked), net_name="auto_attacked_3x.pt", log_name = "auto_attacked_3x")

In [ ]:
train_loader_auto_attacked_5x = DataLoader(dataset=s_auto_attacked, batch_size=64, shuffle=True)

acc_auto_attacked_5x = train_and_evaluate(model_auto_attacked_5x, train_loader_auto_attacked_5x, test_loader, net_reset=(multiplier_sys_reset or M_reset_auto_attacked), net_name="auto_attacked_5x.pt", log_name = "auto_attacked_5x")

In [ ]:
train_loader_auto_attacked_10x = DataLoader(dataset=s_auto_attacked, batch_size=64, shuffle=True)

acc_auto_attacked_10x = train_and_evaluate(model_auto_attacked_2x, train_loader_auto_attacked_10x, test_loader, net_reset=(multiplier_sys_reset or M_reset_auto_attacked), net_name="auto_attacked_10x.pt", log_name = "auto_attacked_10x")

In [ ]:
# Visualize the test accuracy
plt.figure(figsize=(10, 5))

plt.plot(acc_res, marker='o', linestyle='-', markersize=5, label='ResNet_Default')
plt.plot(acc_res_1, marker='o', linestyle='-', markersize=5, label='Poisoned')
plt.plot(acc_auto_attack, marker='x', linestyle='-', markersize=5, label='auto_attacked')
plt.plot(acc_auto_attacked_2x, marker='+', linestyle='-', markersize=5, label='auto_attacked_2x')
plt.plot(acc_auto_attacked_3x, marker='+', linestyle='-', markersize=5, label='auto_attacked_3x')
plt.plot(acc_auto_attacked_5x, marker='+', linestyle='-', markersize=5, label='auto_attacked_5x')
plt.plot(acc_auto_attacked_10x, marker='+', linestyle='-', markersize=5, label='auto_attacked_10x')
# plt.plot(acc_unet_10, marker='x', linestyle='-', markersize=5, label='unet_10')
# plt.plot(acc_auto_attack, marker='x', linestyle='-', markersize=5, label='auto_attack')
# plt.plot(acc_auto_attack, marker='x', linestyle='-', markersize=5, label='unet_attack')
# plt.plot(acc_auto_100, marker='x', linestyle='-', markersize=5, label='auto_100')
# plt.plot(acc_unet_100, marker='x', linestyle='-', markersize=5, label='unet_100')

plt.title('Model Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')

plt.grid(True)
plt.legend()
plt.show()

#### unet_attacked

In [ ]:
train_loader_unet_attacked_2x = DataLoader(dataset=s_unet_attacked, batch_size=64, shuffle=True)

acc_unet_attacked_2x = train_and_evaluate(model_unet_attacked_2x, train_loader_unet_attacked_2x, test_loader, net_reset=(multiplier_sys_reset or M_reset_unet_attacked), net_name="unet_attacked_2x.pt", log_name = "unet_attacked_2x")

In [ ]:
train_loader_unet_attacked_3x = DataLoader(dataset=s_unet_attacked, batch_size=64, shuffle=True)

acc_unet_attacked_3x = train_and_evaluate(model_unet_attacked_3x, train_loader_unet_attacked_3x, test_loader, net_reset=(multiplier_sys_reset or M_reset_unet_attacked), net_name="unet_attacked_3x.pt", log_name = "unet_attacked_3x")

In [ ]:
train_loader_unet_attacked_5x = DataLoader(dataset=s_unet_attacked, batch_size=64, shuffle=True)

acc_unet_attacked_5x = train_and_evaluate(model_unet_attacked_5x, train_loader_unet_attacked_5x, test_loader, net_reset=(multiplier_sys_reset or M_reset_unet_attacked), net_name="unet_attacked_5x.pt", log_name = "unet_attacked_5x")

In [ ]:
train_loader_unet_attacked_10x = DataLoader(dataset=s_unet_attacked, batch_size=64, shuffle=True)

acc_unet_attacked_10x = train_and_evaluate(model_unet_attacked_2x, train_loader_unet_attacked_10x, test_loader, net_reset=(multiplier_sys_reset or M_reset_unet_attacked), net_name="unet_attacked_10x.pt", log_name = "unet_attacked_10x")

In [ ]:
# Visualize the test accuracy
plt.figure(figsize=(10, 5))

plt.plot(acc_res, marker='o', linestyle='-', markersize=5, label='ResNet_Default')
plt.plot(acc_res_1, marker='o', linestyle='-', markersize=5, label='Poisoned')
plt.plot(acc_unet_attack, marker='x', linestyle='-', markersize=5, label='unet_attacked')
plt.plot(acc_unet_attacked_2x, marker='+', linestyle='-', markersize=5, label='unet_attacked_2x')
plt.plot(acc_unet_attacked_3x, marker='+', linestyle='-', markersize=5, label='unet_attacked_3x')
plt.plot(acc_unet_attacked_5x, marker='+', linestyle='-', markersize=5, label='unet_attacked_5x')
plt.plot(acc_unet_attacked_10x, marker='+', linestyle='-', markersize=5, label='unet_attacked_10x')
# plt.plot(acc_unet_10, marker='x', linestyle='-', markersize=5, label='unet_10')
# plt.plot(acc_auto_attack, marker='x', linestyle='-', markersize=5, label='auto_attack')
# plt.plot(acc_auto_attack, marker='x', linestyle='-', markersize=5, label='unet_attack')
# plt.plot(acc_auto_100, marker='x', linestyle='-', markersize=5, label='auto_100')
# plt.plot(acc_unet_100, marker='x', linestyle='-', markersize=5, label='unet_100')

plt.title('Model Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')

plt.grid(True)
plt.legend()
plt.show()

#### auto_100

In [ ]:
train_loader_auto_100_2x = DataLoader(dataset=s_auto_100, batch_size=64, shuffle=True)

acc_auto_100_2x = train_and_evaluate(model_auto_100_2x, train_loader_auto_100_2x, test_loader, net_reset=(multiplier_sys_reset or M_reset_auto_100), net_name="auto_100_2x.pt", log_name = "auto_100_2x")

In [ ]:
train_loader_auto_100_3x = DataLoader(dataset=s_auto_100, batch_size=64, shuffle=True)

acc_auto_100_3x = train_and_evaluate(model_auto_100_3x, train_loader_auto_100_3x, test_loader, net_reset=(multiplier_sys_reset or M_reset_auto_100), net_name="auto_100_3x.pt", log_name = "auto_100_3x")

In [ ]:
train_loader_auto_100_5x = DataLoader(dataset=s_auto_100, batch_size=64, shuffle=True)

acc_auto_100_5x = train_and_evaluate(model_auto_100_5x, train_loader_auto_100_5x, test_loader, net_reset=(multiplier_sys_reset or M_reset_auto_100), net_name="auto_100_5x.pt", log_name = "auto_100_5x")

In [ ]:
train_loader_auto_100_10x = DataLoader(dataset=s_auto_100, batch_size=64, shuffle=True)

acc_auto_100_10x = train_and_evaluate(model_auto_100_2x, train_loader_auto_100_10x, test_loader, net_reset=(multiplier_sys_reset or M_reset_unet_10), net_name="auto_100_10x.pt", log_name = "auto_100_10x")

In [ ]:
# Visualize the test accuracy
plt.figure(figsize=(10, 5))

plt.plot(acc_res, marker='o', linestyle='-', markersize=5, label='ResNet_Default')
plt.plot(acc_res_1, marker='o', linestyle='-', markersize=5, label='Poisoned')
plt.plot(acc_auto_100, marker='x', linestyle='-', markersize=5, label='auto_100')
plt.plot(acc_auto_100_2x, marker='+', linestyle='-', markersize=5, label='auto_100_2x')
plt.plot(acc_auto_100_3x, marker='+', linestyle='-', markersize=5, label='auto_100_3x')
plt.plot(acc_auto_100_5x, marker='+', linestyle='-', markersize=5, label='auto_100_5x')
plt.plot(acc_auto_100_10x, marker='+', linestyle='-', markersize=5, label='auto_100_10x')
# plt.plot(acc_unet_10, marker='x', linestyle='-', markersize=5, label='unet_10')
# plt.plot(acc_auto_attack, marker='x', linestyle='-', markersize=5, label='auto_attack')
# plt.plot(acc_auto_attack, marker='x', linestyle='-', markersize=5, label='unet_attack')
# plt.plot(acc_auto_100, marker='x', linestyle='-', markersize=5, label='auto_100')
# plt.plot(acc_unet_100, marker='x', linestyle='-', markersize=5, label='unet_100')

plt.title('Model Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')

plt.grid(True)
plt.legend()
plt.show()

#### unet_100

In [ ]:
train_loader_unet_100_2x = DataLoader(dataset=s_unet_100, batch_size=64, shuffle=True)

acc_unet_100_2x = train_and_evaluate(model_unet_100_2x, train_loader_unet_100_2x, test_loader, net_reset=(multiplier_sys_reset or M_reset_unet_100), net_name="unet_100_2x.pt", log_name = "unet_100_2x")

In [ ]:
train_loader_unet_100_3x = DataLoader(dataset=s_unet_100, batch_size=64, shuffle=True)

acc_unet_100_3x = train_and_evaluate(model_unet_100_3x, train_loader_unet_100_3x, test_loader, net_reset=(multiplier_sys_reset or M_reset_unet_100), net_name="unet_100_3x.pt", log_name = "unet_100_3x")

In [ ]:
train_loader_unet_100_5x = DataLoader(dataset=s_unet_100, batch_size=64, shuffle=True)

acc_unet_100_5x = train_and_evaluate(model_unet_100_5x, train_loader_unet_100_5x, test_loader, net_reset=(multiplier_sys_reset or M_reset_unet_100), net_name="unet_100_5x.pt", log_name = "unet_100_5x")

In [ ]:
train_loader_unet_100_10x = DataLoader(dataset=s_unet_100, batch_size=64, shuffle=True)

acc_unet_100_10x = train_and_evaluate(model_unet_100_2x, train_loader_unet_100_10x, test_loader, net_reset=(multiplier_sys_reset or M_reset_unet_100), net_name="unet_100_10x.pt", log_name = "unet_100_10x")

In [ ]:
# Visualize the test accuracy
plt.figure(figsize=(10, 5))

plt.plot(acc_res, marker='o', linestyle='-', markersize=5, label='ResNet_Default')
plt.plot(acc_res_1, marker='o', linestyle='-', markersize=5, label='Poisoned')
plt.plot(acc_unet_100, marker='x', linestyle='-', markersize=5, label='unet_100')
plt.plot(acc_unet_100_2x, marker='+', linestyle='-', markersize=5, label='unet_100_2x')
plt.plot(acc_unet_100_3x, marker='+', linestyle='-', markersize=5, label='unet_100_3x')
plt.plot(acc_unet_100_5x, marker='+', linestyle='-', markersize=5, label='unet_100_5x')
plt.plot(acc_unet_100_10x, marker='+', linestyle='-', markersize=5, label='unet_100_10x')
# plt.plot(acc_unet_10, marker='x', linestyle='-', markersize=5, label='unet_10')
# plt.plot(acc_auto_attack, marker='x', linestyle='-', markersize=5, label='auto_attack')
# plt.plot(acc_auto_attack, marker='x', linestyle='-', markersize=5, label='unet_attack')
# plt.plot(acc_auto_100, marker='x', linestyle='-', markersize=5, label='auto_100')
# plt.plot(acc_unet_100, marker='x', linestyle='-', markersize=5, label='unet_100')

plt.title('Model Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')

plt.grid(True)
plt.legend()
plt.show()